In [4]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf

import pip
pip.main(['install', 'xgboost'])
import xgboost as xgb

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [8]:
bat_charge_path = r"/home/yamukelani/source_code/msc_research/rul_prediction/src/datasets/all_batteries_charging_data.csv"
bat_discharge_path = r"/home/yamukelani/source_code/msc_research/rul_prediction/src/datasets/all_batteries_discharging_data.csv"

battery_charging = pd.read_csv(bat_charge_path)
battery_discharging = pd.read_csv(bat_discharge_path)

In [6]:
bat_discharge_sel = battery_discharging[['Ambient_temperature', 'Voltage_measured',
       'Current_measured', 'Temperature_measured', 'Current_load',
       'Voltage_load', 'Discharge_time', 'Capacity']].set_index('Discharge_time')
bat_discharge_sel


,Ambient_temperature,Voltage_measured,Current_measured,Temperature_measured,Current_load,Voltage_load,Capacity
Discharge_time,,,,,,,
0.000,24,4.191492,-0.004902,24.330034,-0.0006,0.000,1.856487
16.781,24,4.190749,-0.001478,24.325993,-0.0006,4.206,1.856487
35.703,24,3.974871,-2.012528,24.389085,-1.9982,3.062,1.856487
53.781,24,3.951717,-2.013979,24.544752,-1.9982,3.030,1.856487
71.922,24,3.934352,-2.011144,24.731385,-1.9982,3.011,1.856487
...,...,...,...,...,...,...,...
2686.359,24,3.443760,-0.002426,35.383979,0.0006,0.000,1.341051
2700.546,24,3.453271,-0.000981,35.179732,0.0006,0.000,1.341051
2714.640,24,3.461963,0.000209,34.977000,0.0006,0.000,1.341051


# Split the battery discharging data into training and test datasets. 

In [ ]:
bat_discharge_train = 